In [134]:
with open('glk.txt') as f:
    text = f.readlines()

In [135]:
g_corpus = '\n'.join(text)

In [213]:
def sent_tokenize(text):
    split = text.split('\n')
    non_empty = filter(lambda sen: sen != '', split)
    return list(non_empty)
# text = 'this is line one\n\n\n\nonline two\n\n\n\n'
# sent_tokenize(text) # result: ['this is line one', 'online two']

In [211]:
# char_in = lambda begin, end: (lambda x : ord(begin) <= ord(x) <= ord(end))
# is_num_char = lambda x: char_in('۰', '۹')(x)
# is_gilaki_char = lambda x: char_in('آ', 'ى')(x) or x in ['چ', 'گ', 'ک', 'ئ', 'ە', 'ی']
# set(filter(is_gilaki_char, list(corpus)))

In [120]:
def get_char_freq(corpus, cond=None):
    chars = {}
    for char in list(corpus):
        if cond is not None:
            if not cond(char):
                continue
        if char not in chars:
            chars[char] = 0

        chars[char] += 1
    
    return sorted(chars.items(), key=lambda t: t[1], reverse=True)

In [238]:
with open('ckb.txt') as f:
    text = f.readlines()
s_corpus = '\n'.join(text)

In [260]:
sorani_special_chars = ['ێ', 'ۆ', 'ئ', 'ڵ', 'ڕ', 'ڤ']
# sorani_special_chars_to_shared_script
sor_conv = {
    'ڤ':'و',
    'ڕ':'ر',
    'ڵ':'ل',
    'ئ':'ئ',
    'ۆ':'ؤ',
    'وو':'ۊ'
}

import re

class Sorani:
    a = 'ە'
    e = 'ێ'
    aa = 'ا'
    
    @staticmethod
    def is_u_vowel(word, index):
        # چک میکنه که آیا در کلمه کردی مورد نظر، و صامت هست یا مصوت
        if index == 0:
            return False
        if word[index-1] in [Sorani.a, Sorani.e, Sorani.aa]:
            return False
        return True
    
def gen_possible_words(w, i=0, res=''):
# convert kurdish word to possible alternatve written candidates
    if i == len(w):
        return [res]
    if w[i] == 'و':
        if Sorani.is_u_vowel(w, i):    
            return gen_possible_words(w, i+1, res+'ؤ') + gen_possible_words(w, i+1, res)
        else:
            return gen_possible_words(w, i+1, res+'و')
    if w[i] == 'ە':
        return gen_possible_words(w, i+1, res+'أ') + gen_possible_words(w, i+1, res)
    return gen_possible_words(w, i+1, res+w[i])
        
# from functools import cache
# @cache
def reduce_sorani_word(w):
# converts sorani word to a word similar to gilaki
    # replace 'وو'
    w = re.sub('وو', sor_conv['وو'], w)
    # replace special chars
    f = lambda c: sor_conv[c] if c in sor_conv else c
    w = ''.join(map(f, list(w)))
        
    word_start_conv = {
        'ئە':'أ',
        'ئا':'آ',
        'ئێ':'ا',
        'ئی':'اي'
    }
    if w[0:2] in word_start_conv:
        w = word_start_conv[w[0:2]] + w[2:]

    return gen_possible_words(w)

In [228]:
def is_word(word, alphabet):
    if len(list(filter(lambda c: c not in alphabet, list(word)))) > 0:
        return False
    return True

sorani_letters = """
ب پ ت د ک گ ق ا ف ڤ س ز خ ع غ ش ژ چ ج ح ھ م ن و ی ر ڕ ل ڵ
ى - ێ ه ە و ۆ ا
ئ
""".split()

def is_sorani(word):
    return is_word(word, sorani_letters)

gilaki_letters = """
 ٚ آ أ ا ب پ ت ط ث س ص ج چ ح ه خ د ذ ز ض ظ ر ژ ش ع غ ق ف ک گ ل م ن و ۊ ؤ ی ي ئ 
""".split() + ['-', '‌']

def is_gilaki(word):
    return is_word(word, gilaki_letters)

In [214]:
# for word in corpus.split()[0:20]:
#     print(word, reduce_sorani_word(word), is_sorani(word))

In [223]:
def words_containing(letter, words):
    return list(set(filter(lambda w: letter in w, words)))

In [239]:
words_containing('ئێ', s_corpus.split())[0:10]

['ئێرانیی',
 'ئێستە.',
 '(ئێرانی',
 'ئێبی،',
 'ئێڵەکان',
 'ئێرانییەکانی',
 'ئێران)،',
 'ئێستا',
 'ئێستادا',
 'ئێرانمیانمار']

In [240]:
words_containing('أ', g_corpus.split())[0:10]

['خۊرتأو',
 'أمره',
 'أنه',
 'خؤسأنئن',
 'دأنه',
 'بۊجؤر-أ',
 'دأشت،',
 'ميان\u200cپوشته\u200cأن',
 'أمنيت',
 'ونأشت']

In [326]:
import numpy as np

def MED(a, b):
    # minimal edit distance
    n, m = len(a), len(b)
    M = np.zeros((n+1, m+1))#.astype('int')
    
    M[:,0] = np.arange(n+1)
    M[0,:] = np.arange(m+1)
    
    def sub(v,w):
        if v==w: 
            return 0
        U = ['ؤ', 'ۊ', 'و']
        S = ['ث', 'ص', 'س']
        T = ['ط','ت']
        Z = ['ز','ض','ذ','ظ']
        V = ['ي', 'ئ', 'أ', 'ا'] # vowels, needs more thought
        
        for closeset in [U,S,T,Z,V]:
            if v in closeset and w in closeset:
                return 0.25

        return 2
        
    for i in range(n):
        for j in range(m):
            M[i+1,j+1] = min(
                M[i,j+1]+1, M[i+1,j]+1,
                M[i,j] + sub(a[i], b[j])
            )

    distance = M[n,m]
    return distance

In [272]:
import re
def normalize(text):
    letter_map = {
       '\u200c': '‌',
        'ك':'ک',
        'ة':'ه'
    }
    return ''.join([letter_map.get(t, t) for t in list(text)])
    
def extract_words(text, alphabet):
    text = normalize(text)
    cleaned_text = ''.join([t if t in alphabet else ' ' for t in list(text)])
    return list(set(cleaned_text.split()))
extract_words(g_corpus, gilaki_letters)[0:10]

['هرتسفیلد',
 'اسفراين',
 'تیج',
 'سلبس',
 'ئن',
 'چاله\u200cمیدان',
 'کارگيرؤنه',
 'میاندوآب',
 'حتما',
 'دامغان-ه']

In [273]:
sorani_words = extract_words(s_corpus, sorani_letters)
gilaki_words = extract_words(g_corpus, gilaki_letters)

In [307]:
sor_words = list(filter(lambda w: len(w)>2, sorani_words))
glk_words = list(filter(lambda w: len(w)>2, gilaki_words))

In [312]:
len(sor_words), len(glk_words)

(17240, 7128)

In [338]:
def distance_sor_glk(sor, glk):
    return min([(MED(word, glk),word) for word in reduce_sorani_word(sor)])

dists = []

from tqdm import tqdm

for sor in tqdm(sor_words[:]):
    for glk in glk_words[:]:
#         d = distance_sor_glk(sor, glk)
        d = (MED(glk, sor), sor, glk)
        if d[0] < 2:
            print(d)
            dists.append(d)

  0%|          | 2/17240 [00:01<2:23:31,  2.00it/s]

(1.0, 'وسه', 'وسته')


  0%|          | 3/17240 [00:01<2:21:40,  2.03it/s]

(1.25, 'وسه', 'تؤسه')


  0%|          | 8/17240 [00:07<5:09:59,  1.08s/it]

...

(1.0, 'سنوری', 'نوری')


 23%|██▎       | 3940/17240 [1:07:20<4:32:10,  1.23s/it]

(1.5, 'ئیمپراتوری', 'امپراطوری')
(1.25, 'ئیمپراتوری', 'امپراتوری')
(1.0, 'گورجی', 'گرجی')


 23%|██▎       | 3941/17240 [1:07:21<3:47:17,  1.03s/it]


KeyboardInterrupt: 

In [317]:
# import heapq
# best = heapq.nsmallest(30, dists)

In [337]:
dists

[(1.25, 'داڕشت', 'دأشت'),
 (1.25, 'وته', 'سۊته'),
 (1.0, 'وته', 'وسته'),
 (0.25, 'شاره', 'شأره'),
 (1.0, 'شاره', 'شماره'),
 (1.0, 'شاره', 'اشاره'),
 (1.0, 'کوردی', 'کردی'),
 (1.0, 'فارسی', 'فارس'),
 (1.0, 'راگه', 'راه'),
 (1.0, 'چیتای', 'چیتی'),
 (1.0, 'سار', 'ساری'),
 (1.25, 'سار', 'صارا'),
 (1.25, 'دانس', 'دأن'),
 (1.25, 'دانس', 'دئن'),
 (1.25, 'دانس', 'دين'),
 (1.0, 'جوو', 'جولو'),
 (1.25, 'دوان', 'دأن'),
 (1.25, 'دوان', 'دئن'),
 (1.0, 'دوان', 'دیوان'),
 (1.25, 'دوان', 'دين'),
 (1.0, 'سێفید', 'سفید'),
 (1.25, 'ئلی', 'هالی'),
 (1.25, 'ئلی', 'خالی'),
 (1.0, 'ماما', 'اما'),
 (1.25, 'ماما', 'ميم'),
 (1.25, 'وان', 'ۊنان'),
 (1.25, 'وان', 'ويني'),
 (1.0, 'دامه', 'دام'),
 (1.0, 'باشور', 'باور'),
 (1.0, 'بەکار', 'بکار'),
 (1.0, 'دوو', 'دوبو'),
 (1.0, 'بری', 'برئی'),
 (1.0, 'بری', 'برخی'),
 (1.0, 'کۆردی', 'کردی'),
 (1.25, 'ئیریان', 'ایران'),
 (1.0, 'قودسی', 'قدسی'),
 (1.0, 'مادی', 'ماد'),
 (1.25, 'داوا', 'ديوار'),
 (1.0, 'بونی', 'بون'),
 (1.0, 'کول', 'کولی'),
 (1.25, 'تورکی', 'تۊرکیه'),
 (1.